## Diferenciación automática con autograd

**source**: http://datahacker.rs/004-computational-graph-and-autograd-with-pytorch/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch

from IPython.display import display, Math

In [ ]:
#!pip install torchviz
from torchviz import make_dot

# EJEMPLO 1

#### $$\widehat{{y}} = w \cdot x + b$$
#### $$w=2, \;\; b=1 \;\; \Rightarrow \;\; \mathcal{{L}} = ([2x + 1] - y)^{{2}}$$

#### $$u = 2 \cdot x$$
#### $$y_{{p}} = u + 1$$
#### $$e = y_{{p}} - y$$
#### $$\mathcal{{L}} = e^{{2}}$$
---

<div>
<center><img style="width:800px;" src="data:image/jpeg;base64, ..."/>
</center>
</div>

In [ ]:
x = torch.tensor(3., requires_grad=True)
y = torch.tensor(5., requires_grad=True)
b = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)

u = x * w
u.retain_grad()  # Retiene el gradiente intermedio

yp = u + b
yp.retain_grad()  # Retiene el gradiente intermedio

e = yp - y
e.retain_grad()  # Retiene el gradiente intermedio

L = e ** 2
L.retain_grad()  # Retiene el gradiente intermedio

print(L)

In [ ]:
L.backward(retain_graph=True)  # True --> Conserva grafo y gradientes

In [ ]:
w.grad

In [ ]:
y.grad

# EJEMPLO 2

## $$y = 3x^{2}+4x+2$$
---

<div>
<center><img style="width:800px;" src="data:image/jpeg;base64, ..."/>
</center>
</div>

In [ ]:
x = torch.tensor(3., requires_grad=True)

In [ ]:
y = 3 * x**2 + 4 * x + 2

print(y)

In [ ]:
y.backward()  # Calculo gradientes

display(Math(f'y = {y}'))

display(Math(fr'\frac{{\partial y}}{{\partial x}} = {x.grad.item()}'))

In [ ]:
make_dot(y,
         params={'x':x, 'y': y},
         show_attrs=True,
         show_saved=True)

### Deshabilitar gradiente

In [ ]:
w = x.clone()

print(id(x), id(w))

In [ ]:
print(x)

print(x.detach())

print(x)

In [ ]:
print(x)

x.requires_grad = False
#x.requires_grad_(False)

print(x)

### Acumulación de gradiente

In [ ]:
x = torch.tensor(3., requires_grad=True)

for epoch in range(3):
    
    y = 3*x**2 + 4*x + 2
    
    y.backward()
    
    print(x.grad)
    #x.grad.zero_()